## Spatial Data Science with CityJSON

The purpose of this Notebook is to ***work with*** the product of [osm_LoD1_3DCityModel](https://github.com/AdrianKriger/osm_LoD1_3DCityModel); a previously created CityJSON city model.

**This notebook will:**

> **1. allow the user to execute an application of Spatial Data Science**  
>
>> **a)  population estimation and**  
>> **b)  a measure of [Building Volume per Capita](https://www.researchgate.net/publication/343185735_Building_Volume_Per_Capita_BVPC_A_Spatially_Explicit_Measure_of_Inequality_Relevant_to_the_SDGs).**
>
> **2. an interactive visualization** *-via [pydeck](https://deckgl.readthedocs.io/en/latest/)- which a user can navigate, query and share*.  

In [1]:
#load the magic

%matplotlib inline
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, shape, mapping
import json
import geojson

from cjio import cityjson

import matplotlib.pyplot as plt
import pydeck as pdk

**The area under investigation is [University Estate, Cape Town. South Africa](https://en.wikipedia.org/wiki/University_Estate). Its 3D.CityJSON is available as `citjsnClean_uEstate10m.json` in the [result folder](https://github.com/AdrianKriger/osm_LoD1_3DCityModel/blob/main/village_campus/result/citjsnClean_uEstate10m.json)**

In [8]:
#- use the parameter file from osm_LoD1_3DCityModel ~ osm3DuEstate_param.json, etc. [extra folder] 
#- and change the "cjsn_solid": "./result/citjsnClean_uEstate3D.json" to 
#-                "cjsn_solid": "./result/citjsnClean_uEstate10m.json" 

jparams = json.load(open('osm3DuEstate_param.json'))

In [9]:
cm = cityjson.load(path=jparams['cjsn_solid']) #-- citjsnClean_uEstate10m.json in the result folder

In [10]:
print(cm)

CityJSON version = 1.1
EPSG = None
bbox = [ 264030.588 6241349.216 45.510 265156.692 6242338.955 167.920 ]
=== CityObjects ===
|-- TINRelief (1)
|-- Building (305)
materials = False
textures = False


In [11]:
df = cm.to_dataframe()
#- remove the first feature: the terrain
df = df[1:]            

gdf = gpd.GeoDataFrame(df, geometry=[shape(d) for d in df.pop("footprint")], crs=jparams['crs'])
#gdf.head(2)

## 1. Spatial Data Science

<div class="alert alert-block alert-warning"><b>We start with basic spatial analysis</b>  
    
     
- We'll estimate the population, within our area of interest, and then  
- calculate the Building Volume Per Capita (BVPC).
</div>

While estimating population is well documented; recent investigations to **understand overcrowding** have led to newer measurements.  

The most noteable of these is **Building Volume Per Capita (BVPC)** [(Ghosh, T; et al. 2020)](https://www.researchgate.net/publication/343185735_Building_Volume_Per_Capita_BVPC_A_Spatially_Explicit_Measure_of_Inequality_Relevant_to_the_SDGs). BVPC is the cubic meters of building per person. **BVPC tells us how much space one person has per residential living unit** (a house / apartment / etc.). It is ***a proxy measure of economic inequality and a direct measure of housing inequality***.

BVPC builds on the work of [(Reddy, A and Leslie, T.F., 2013)](https://www.tandfonline.com/doi/abs/10.1080/02723638.2015.1060696?journalCode=rurb20) and attempts to integrate with several **[Sustainable Development Goals](https://sdgs.un.org/goals)** (most noteably: **[SDG 11: Developing sustainable cities and communities](https://sdgs.un.org/goals/goal11)**) and captures the average ***'living space'*** each person has in their home.

<div class="alert alert-block alert-info"><b>These analysis expect the user to have some basic knowledge about the environment under inquiry / investigation</b> </div>

In [12]:
gdf.head(2)

,osm_id,osm_address,osm_building,osm_building:levels,plus_code,ground_height,building_height,roof_height,osm_name,osm_office,osm_type,osm_website,osm_operator,geometry
739615941,739615941.0,10 Rhodes Avenue University Estate Cape Town,house,2,4FRW3C6X+WRG,95.95,6.9,102.85,NaN,NaN,NaN,NaN,NaN,"POLYGON ((264275.999 6241813.835, 264278.104 6..."
740820432,740820432.0,100 Upper Roodebloem Road University Estate Ca...,house,2,4FRW3F62+R87,85.20,6.9,92.10,NaN,NaN,NaN,NaN,NaN,"POLYGON ((264379.703 6241790.620, 264379.769 6..."


In [13]:
#gdf.plot()

<div class="alert alert-block alert-success"><b>1.  a) Estimate Population:</b></div>

In [14]:
#--we only want building=house or =apartment or =residential
gdf_pop = gdf[gdf["osm_building"].isin(['house', 'apartment', 'residential'])].copy()

In [15]:
len(gdf_pop)

295

**This area is urban with single level housing units. To estimate population is thus pretty straight forward.**

<div class="alert alert-block alert-info"><b>We start with local knowledge.</b></div>

**On average there are roughly `4` people per `building:house` in this area.**  

**Additionally an *informal* structure is tagged `building:residential` and houses `3` people.**

<div class="alert alert-block alert-warning"><b></b>  
    
**Furthermore:**  
    - `building:apartment` harvests the `building:flats` *'key:value'* pair *(the number of units)* to calculate `*3` people per apartment.  
    - Student accomodation is tagged `building:residential` with `residential:student` and then harvests the `building:flats` *'key:value'* pair *(the number of units)* to calculate `*1` people per apartment; if `level: > 1` else `*3` people in a house share.
    
**The tagging scheme and numbers is based on *how your community is mapped* and local knowledge**
</div>

In [16]:
def pop(row):
    if row['osm_building'] == 'house':
        return 4
    if row['osm_building'] == 'apartment':
        return row['flats'] * 3
    if row['osm_building'] == 'residential': #here should be an additional: and row['res'] == 'informal':
        return 3
    if row['osm_building'] == 'residential' and row['res'] == 'student':
        if row['levels'] > 1:
            return row['flats'] * 1
        else:
            3

gdf_pop['pop'] = gdf_pop.apply(lambda x: pop(x), axis=1)

est_pop = gdf_pop['pop'].sum()
print('The estimated population is:', est_pop)

The estimated population is: 1180


**The official [STATSSA 2011 census figure](https://en.wikipedia.org/wiki/University_Estate), for this community, is 987** and suggests a population growth rate of approximately 1.49% per year.

This growth rate is calculated using the formula for **[Annual population growth](https://databank.worldbank.org/metadataglossary/health-nutrition-and-population-statistics/series/SP.POP.GROW):**

$$r = \frac{\ln{[\frac{End Population}{Start Population}}]}{n} * 100 = \frac{\ln{[\frac{1 180}{987}}]}{12} * 100   = 1.49\%$$


<div class="alert alert-block alert-success"><b>1. b) Building Volume Per Capita (BVPC):</b>  
BVPC = total population of a community divided by sum of building volume</div>

In [17]:
gdf_pop['area'] = gdf_pop['geometry'].area#\.map(lambda p: p.area)
gdf_pop['volume'] = gdf_pop['area'] * gdf_pop['building_height']
gdf_pop['bvpc'] =  gdf_pop['volume'] / gdf_pop['pop']

gdf_pop.tail(2)

,osm_id,osm_address,osm_building,osm_building:levels,plus_code,ground_height,building_height,roof_height,osm_name,osm_office,osm_type,osm_website,osm_operator,geometry,pop,area,volume,bvpc
1025219390,1.025219e+09,10 Kylemore Road University Estate Cape Town,house,2,4FRW3C6X+RJF,104.20,6.9,111.10,NaN,NaN,NaN,NaN,NaN,"POLYGON ((264231.363 6241791.926, 264231.955 6...",4,123.181468,849.952130,212.488032
1025219391,1.025219e+09,2 Rhodes Avenue University Estate Cape Town,house,2,4FRW3C7X+2JJ,101.18,6.9,108.08,NaN,NaN,NaN,NaN,NaN,"POLYGON ((264224.292 6241856.433, 264214.469 6...",4,105.446178,727.578631,181.894658


In [18]:
print(gdf_pop['bvpc'].describe())

count    295.000000
mean     213.642812
std      126.850648
min       34.491596
25%      129.096209
50%      179.227566
75%      265.138697
max      971.830674
Name: bvpc, dtype: float64


In [19]:
bvpc = round(gdf_pop['volume'].sum() / est_pop, 3)

print('Building Volume Per Capita (BVPC):', bvpc)

Building Volume Per Capita (BVPC): 213.643


<div class="alert alert-block alert-info"><b></b>

**This BVPC value is general.**  

We can seperate `building:house` and `building:residential` to undertand the differences between ***formal and informal*** housing in this area.
    
**We want to understand the living space *(the cubic-meter BVPC value)* each person has in thier home**
</div>

In [20]:
formal = gdf_pop[gdf_pop["osm_building"].isin(['house'])].copy()
f_pop = formal['pop'].sum()
#f_area = formal['area'].mean()

informal = gdf_pop[gdf_pop["osm_building"].isin(['residential'])].copy()
inf_pop = informal['pop'].sum()
#inf_area = formal['area'].mean()

bvpc_formal = round(formal['volume'].sum() / est_pop, 3)
bvpc_informal = round(informal['volume'].sum() / est_pop, 3)

print('FORMAL: Population: ', f_pop, ' with Building Volume Per Capita (BVPC):', bvpc_formal)
print('')
print('INFORMAL: Polutation: ', inf_pop, ' with Building Volume Per Capita (BVPC)', bvpc_informal)

FORMAL: Population:  1180  with Building Volume Per Capita (BVPC): 213.643

INFORMAL: Polutation:  0  with Building Volume Per Capita (BVPC) 0.0


## 2. Interactive Visualization

You might want to create and share an `html` visualization.

<div class="alert alert-block alert-warning"><b></b>  
    
_In this example we identify building stock by **color** but you are limited only through your imagination and the data you have access too_
</div>

In [21]:
#- pydeck needs geographic coords
gdf = gdf.to_crs(4326)

In [ ]:
# -- get the location for pydeck
[xy] = gdf.dissolve().centroid

bbox = [gdf.total_bounds[0], gdf.total_bounds[1], 
        gdf.total_bounds[2], gdf.total_bounds[3]]

In [23]:
# have a look at the building type and amenities available
gdf['osm_building'].unique()

array(['house', 'garage', 'yes', 'office', 'warehouse'], dtype=object)

In [24]:
# colour buildings based on use / amenity
def color(bld):
    if bld == 'house':# or bld == 'yes':
        return [255, 255, 204]
    if bld == 'retail' or bld == 'office':
        return [253, 141, 60]
    if bld == 'school':
        return [128, 0, 38]
    if bld == 'clinic' or bld == 'doctors':
        return [89, 182, 178]
    if bld == 'community_centre' or bld == 'service' or bld ==  'post_office' \
    or bld ==  'townhall' or bld ==  'police':
        return [181, 182, 89]
    if bld == 'library':
        return [193, 255, 193]
    if bld == 'restaurant':
        return [139, 117, 0]
    if bld == 'place_of_worship':
        return [225, 225, 51]
    if bld == 'warehouse':
        return [51, 255, 94]
    else:
        return [255, 255, 204]

gdf["fill_color"] = gdf['osm_building'].apply(lambda x: color(x))

In [25]:
#- look
gdf.head(2)

,osm_id,osm_address,osm_building,osm_building:levels,plus_code,ground_height,building_height,roof_height,osm_name,osm_office,osm_type,osm_website,osm_operator,geometry,fill_color
739615941,739615941.0,10 Rhodes Avenue University Estate Cape Town,house,2,4FRW3C6X+WRG,95.95,6.9,102.85,NaN,NaN,NaN,NaN,NaN,"POLYGON ((18.44961 -33.93777, 18.44964 -33.937...","[255, 255, 204]"
740820432,740820432.0,100 Upper Roodebloem Road University Estate Ca...,house,2,4FRW3F62+R87,85.20,6.9,92.10,NaN,NaN,NaN,NaN,NaN,"POLYGON ((18.45073 -33.93800, 18.45073 -33.938...","[255, 255, 204]"


In [26]:
## ~ (x, y) - bl, tl, tr, br  ~~ or ~~ sw, nw, ne, se
#area = [[[18.4377, -33.9307], [18.4377, -33.9283], [18.4418, -33.9283], [18.4418, -33.9307]]]
area = [[[bbox[0], bbox[1]], [bbox[0], bbox[3]], 
         [bbox[2], bbox[3]], [bbox[2], bbox[1]]]]

## ~ (y, x)
view_state = pdk.ViewState(latitude=xy.y, longitude=xy.x, zoom=16.5, max_zoom=19, pitch=72, 
                                   bearing=80)

land = pdk.Layer(
    "PolygonLayer",
    area,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[0, 0, 0, 1],
    #material = True,
    #shadowEnabled = True
)
building_layer = pdk.Layer(
    "PolygonLayer",
    gdf,
    #id="geojson",
    opacity=0.3,
    stroked=False,
    get_polygon="geometry.coordinates",
    filled=True,
    extruded=True,
    wireframe=False,
    get_elevation="building_height",
    #get_fill_color="[255, 255, 255]", #255, 255, 255
    get_fill_color="fill_color",
    get_line_color="fill_color",#[255, 255, 255],
    #material = True, 
    #shadowEnabled = True, 
    auto_highlight=True,
    pickable=True,
)

tooltip = {"html": "<b>Levels:</b> {osm_building:levels} <br/> <b>Address:</b> {osm_address}\
<br/> <b>Plus Code:</b> {plus_code} <br/> <b>Building Type:</b> {osm_building}"}

#change the tooltip to show bus routes and comment out the previous
#tooltip = {"html": "<b>Route:</b> {name} <br/>"}

r = pdk.Deck(layers=[land,  building_layer],#, greenspaces_layer, p_layer, water_layer, r_layer], #
             #views=[{"@@type": "MapView", "controller": True}],
             initial_view_state=view_state,
             map_style = 'dark_no_labels', #pdk.map_styles.LIGHT,
             tooltip=tooltip)
#save
r.to_html("./result/interactiveOnly.html")

**on a laptop without a mouse:**

- `trackpad left-click drag-left` and `-right`;
- `Ctrl left-click drag-up`, `-down`, `-left` and `-right` to rotate and so-on and
- `+` next to Backspace zoom-in and `-` next to `+` zoom-out.

**Now you do your community.** ~ If your area needs [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap)  data and you want to contribute please follow the [Guide](https://wiki.openstreetmap.org/wiki/Beginners%27_guide).